# Anti-Cancer Peptide Prediction with Machine Learning
### University Machine Learning Intro Course: Final Project

This project uses peptide sequence data (in FASTA format) to train a binary classifier predicting anticancer peptide (ACP) activity. Data was provided in a course context and is not included here due to licensing.

## The Brief

Anticancer peptides (ACPs) are short chains of amino acids, typically 5 to 50
residues long, that can selectively target and kill cancer cells while minimizing harm to normal cells. They often work by disrupting cancer cell membranes or triggering apoptosis and are less prone to resistance than traditional drugs. ACPs can be naturally derived or synthetically designed, making them promising candidates for targeted cancer therapies.

In computational biology, ACPs are represented in the FASTA format using
standard single-letter amino acid codes. With 20 standard amino acids, the
number of possible sequences grows exponentially with length, making the
identification and design of effective ACPs a complex but promising area of
research.

In this project, I trained a model that predicts whether a peptide has anticancer activity based on its FASTA sequence. I trialled a few models which I discuss in the conclusion, and ultimately implemented a bidirectional GRU model in the final pipeline.

I assessed AUC metrics ROC AUC and PR AUC, particularly focussing on the means across seeds and the standard deviation score of ROC AUC.

In [5]:
import os
import random
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, average_precision_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Bidirectional, GRU, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
# Parameters
MAX_LEN = 50   # Verified in EDA
ACIDS = "ACDEFGHIKLMNPQRSTVWY"   # The 20 amino acids present in the peptides
BATCH_SIZE = 32
EPOCHS = 15

In [7]:
# Set seeds for reproducibility
def set_seeds(seed):
  os.environ["PYTHONHASHSEED"] = str(seed)
  random.seed(seed)
  np.random.seed(seed)
  tf.random.set_seed(seed)
  os.environ["TF_DETERMINISTIC_OPS"] = "1"

## Data Loading and Exploration

In [8]:
# Uploading the data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train["FASTA"].values
y = train["label"].values

X_test = test["FASTA"].values
y_test = test["label"].values

In [9]:
# Obtaining types and count
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2352 entries, 0 to 2351
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   FASTA   2352 non-null   object
 1   label   2352 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 36.9+ KB


In [10]:
# Obtaining types and count
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3370 entries, 0 to 3369
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   FASTA   3370 non-null   object
 1   label   3370 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 52.8+ KB


In [11]:
# Assessing sequence lengths
sequence_lengths = []
for sequence in X:
  sequence_lengths.append(len(sequence))

pd.Series(sequence_lengths).describe()

,0
count,2352.000000
mean,24.230867
std,10.239531
min,6.000000
25%,16.000000
50%,22.000000
75%,30.000000
max,50.000000


## Encoding

In [12]:
# One-hot encoding: encoding amino acids as integers
def one_hot_encoder(sequence, length, acids=ACIDS):

  # Indexes the string of the 20 amino acids present in peptide sequences
  indexed_acids = {acid: i for i, acid in enumerate(acids)}

  # A matrix of zeroes for encoding
  encoding = np.zeros((length, len(acids)), dtype=np.float32) # Pad sequences to uniform length

  # Fills a 1 in the matrix where the acid in the sequence matches the enumerated acids string
  for i, acid in enumerate(sequence[:length]):
    if acid in indexed_acids:
      encoding[i, indexed_acids[acid]] = 1.0
    else:
      print(f"Invalid sequence: {sequence}")
      break

  return encoding

## Model Definition

In [13]:
# Bidirectional GRU
def bidirect_GRU(input_shape):
  model = Sequential([
      Bidirectional(GRU(64, return_sequences=True), input_shape=input_shape),
      Bidirectional(GRU(32, return_sequences=False)),
      BatchNormalization(),
      Dense(64, activation='relu'),
      Dropout(0.3),
      Dense(1, activation="sigmoid")
  ])
  model.compile(optimizer="adam", loss="binary_crossentropy", metrics=[tf.keras.metrics.AUC(name="auc"), tf.keras.metrics.AUC(name='pr_auc', curve='PR')])
  return model

## Model Training

In [14]:
roc_aucs = []
pr_aucs = []

X_enc = np.array([one_hot_encoder(seq, MAX_LEN) for seq in X])
X_test_enc = np.array([one_hot_encoder(seq, MAX_LEN) for seq in X_test])

for seed in range(1, 6):
  set_seeds(seed)

  X_train, X_val, y_train, y_val = train_test_split(X_enc, y, test_size=0.2, stratify=y, random_state=seed)

  model = bidirect_GRU(input_shape=(MAX_LEN, len(ACIDS)))
  early_stopping = EarlyStopping(monitor='val_auc', mode='max', patience=2, restore_best_weights=True)
  model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=(X_val, y_val), callbacks=[early_stopping])

  y_pred = model.predict(X_test_enc)

  roc = roc_auc_score(y_test, y_pred)
  pr = average_precision_score(y_test, y_pred)

  roc_aucs.append(roc)
  pr_aucs.append(pr)

  print(f"Seed {seed} ROC AUC: {roc:.4f}, PR AUC: {pr:.4f}")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - auc: 0.6752 - loss: 0.6419 - pr_auc: 0.7001 - val_auc: 0.8130 - val_loss: 0.6588 - val_pr_auc: 0.8282
Epoch 2/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 19ms/step - auc: 0.8269 - loss: 0.5123 - pr_auc: 0.8381 - val_auc: 0.8569 - val_loss: 0.6993 - val_pr_auc: 0.8553
Epoch 3/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step - auc: 0.8623 - loss: 0.4660 - pr_auc: 0.8681 - val_auc: 0.8656 - val_loss: 0.6190 - val_pr_auc: 0.8623
Epoch 4/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - auc: 0.8721 - loss: 0.4507 - pr_auc: 0.8757 - val_auc: 0.8790 - val_loss: 0.5542 - val_pr_auc: 0.8656
Epoch 5/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 40ms/step - auc: 0.8794 - loss: 0.4380 - pr_auc: 0.8842 - val_auc: 0.8791 - val_loss: 0.6175 - val_pr_auc: 0.8625
Epoch 6/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 3s 56ms/step - auc: 0.8864 - loss: 0.4265 - pr_auc: 0.8896 - val_auc: 0.8776 - val_loss: 0.7850 - val_pr_auc: 0.8458
Epoch 7/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 29ms/step - auc: 0.8939 - lo

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


59/59 ━━━━━━━━━━━━━━━━━━━━ 7s 35ms/step - auc: 0.6895 - loss: 0.6675 - pr_auc: 0.6826 - val_auc: 0.7749 - val_loss: 0.6580 - val_pr_auc: 0.7756
Epoch 2/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8277 - loss: 0.5135 - pr_auc: 0.8411 - val_auc: 0.8228 - val_loss: 0.6165 - val_pr_auc: 0.8005
Epoch 3/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8665 - loss: 0.4565 - pr_auc: 0.8756 - val_auc: 0.8377 - val_loss: 0.6195 - val_pr_auc: 0.8112
Epoch 4/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8750 - loss: 0.4431 - pr_auc: 0.8834 - val_auc: 0.8455 - val_loss: 0.6597 - val_pr_auc: 0.8205
Epoch 5/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8854 - loss: 0.4280 - pr_auc: 0.8911 - val_auc: 0.8511 - val_loss: 0.6385 - val_pr_auc: 0.8300
Epoch 6/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8888 - loss: 0.4239 - pr_auc: 0.8950 - val_auc: 0.8484 - val_loss: 0.6979 - val_pr_auc: 0.8235
Epoch 7/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8983 - loss: 0.4066 -

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step - auc: 0.6696 - loss: 0.6504 - pr_auc: 0.6855 - val_auc: 0.8189 - val_loss: 0.6620 - val_pr_auc: 0.8138
Epoch 2/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - auc: 0.8274 - loss: 0.5212 - pr_auc: 0.8311 - val_auc: 0.8633 - val_loss: 0.6282 - val_pr_auc: 0.8643
Epoch 3/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - auc: 0.8714 - loss: 0.4514 - pr_auc: 0.8778 - val_auc: 0.8683 - val_loss: 0.6118 - val_pr_auc: 0.8641
Epoch 4/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8870 - loss: 0.4301 - pr_auc: 0.8922 - val_auc: 0.8735 - val_loss: 0.5444 - val_pr_auc: 0.8749
Epoch 5/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8973 - loss: 0.4100 - pr_auc: 0.9018 - val_auc: 0.8764 - val_loss: 0.5030 - val_pr_auc: 0.8781
Epoch 6/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.9031 - loss: 0.3989 - pr_auc: 0.9056 - val_auc: 0.8667 - val_loss: 0.5011 - val_pr_auc: 0.8686
Epoch 7/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - auc: 0.9107 - loss: 0.3845 -

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


59/59 ━━━━━━━━━━━━━━━━━━━━ 7s 33ms/step - auc: 0.7025 - loss: 0.6313 - pr_auc: 0.7015 - val_auc: 0.7920 - val_loss: 0.6669 - val_pr_auc: 0.8157
Epoch 2/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8221 - loss: 0.5202 - pr_auc: 0.8307 - val_auc: 0.8484 - val_loss: 0.6284 - val_pr_auc: 0.8518
Epoch 3/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - auc: 0.8670 - loss: 0.4563 - pr_auc: 0.8751 - val_auc: 0.8760 - val_loss: 0.5689 - val_pr_auc: 0.8753
Epoch 4/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8846 - loss: 0.4328 - pr_auc: 0.8873 - val_auc: 0.8877 - val_loss: 0.5306 - val_pr_auc: 0.8863
Epoch 5/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8965 - loss: 0.4091 - pr_auc: 0.8981 - val_auc: 0.8873 - val_loss: 0.4733 - val_pr_auc: 0.8898
Epoch 6/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.8969 - loss: 0.4104 - pr_auc: 0.8975 - val_auc: 0.8938 - val_loss: 0.4506 - val_pr_auc: 0.8965
Epoch 7/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.9064 - loss: 0.3883 -

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


59/59 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - auc: 0.7121 - loss: 0.6202 - pr_auc: 0.7085 - val_auc: 0.7866 - val_loss: 0.6703 - val_pr_auc: 0.7832
Epoch 2/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - auc: 0.8461 - loss: 0.4940 - pr_auc: 0.8488 - val_auc: 0.8169 - val_loss: 0.6568 - val_pr_auc: 0.8038
Epoch 3/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - auc: 0.8828 - loss: 0.4349 - pr_auc: 0.8866 - val_auc: 0.8481 - val_loss: 0.6124 - val_pr_auc: 0.8340
Epoch 4/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - auc: 0.8933 - loss: 0.4151 - pr_auc: 0.8942 - val_auc: 0.8509 - val_loss: 0.5832 - val_pr_auc: 0.8365
Epoch 5/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - auc: 0.9044 - loss: 0.3927 - pr_auc: 0.9096 - val_auc: 0.8427 - val_loss: 0.5586 - val_pr_auc: 0.8241
Epoch 6/15
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - auc: 0.9043 - loss: 0.3931 - pr_auc: 0.9063 - val_auc: 0.8382 - val_loss: 0.5487 - val_pr_auc: 0.8112
106/106 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step
Seed 5 ROC AUC: 0.8376, PR AUC: 0.4609


### ROC AUC and PR AUC

The model accuracy requirements are as follows:

*   Mean ROC AUC ≥ 0.83
*   Std ROC AUC ≤ 0.015
*   Mean PR AUC ≥ 0.43

In [15]:
# Mean and standard deviation scores
headings = ["ROC AUC", "PR AUC"]
roc_auc_scores = [np.mean(roc_aucs), np.std(roc_aucs)]
pr_auc_scores = [np.mean(pr_aucs), np.std(pr_aucs)]
pd.DataFrame([roc_auc_scores, pr_auc_scores], columns=["Mean", "Standard Deviation"], index=headings)

,Mean,Standard Deviation
ROC AUC,0.841761,0.009650
PR AUC,0.449067,0.020093


## Conclusion

### Model Choice

Initially, I implemented a 1D CNN model due to its use in detecting local patterns in sequences, but it failed to exceed a mean ROC AUC of 0.83 across 5 seeds despite tuning (kernel size, filter count) and the addition of layers. On paper, a 1D CNN model sounded highly applicable to my task, but in retrospect, it likely suffered from memory limitations and highly localised pattern seeking rather than a more global approach.

I implemented a simple RNN next using only a single bidirectional GRU layer and adding a second when the metrics fell just shy of the requirements. Bidirectional GRUs are highly sequence-aware, position-sensitive, and have greater memory capacity making them well-suited to capturing meaningful patterns in the sequences. Implementing multiple `Bidirectional(GRU())` layers, and a `BatchNormalisation()` layer, was enough to push mean AUC scores over the mark while minimising standard deviation.

In the interest of improving metrics further, I tried a hybrid version of my two models, combining `Conv1D()` with `Bidirectional(GRU())`, but while the difference in means was negligible, the added complexity substantially increased standard deviation beyond the ≤ 0.015 requirement.

### Encoding

I used one-hot encoding because of its simplicity and suitability to simple fixed-length sequences, but with the implementation of an RNN, embedding would have been the more logical option. It would be the first thing I alter about the pipeline in terms of future direction, to see if the relational and adaptive characteristics of embedding improve model performance. That greater contextual awareness would allow for generalisation, treating acids within peptide sequences as interrelated, rather than distinct from one another as one-hot encoding does.